## 確率的試行のシミュレーション

### サンプリング

In [1]:
using Distributions

In [2]:
bern = Bernoulli(0.5)

x = rand(bern, 10)

10-element Vector{Bool}:
 1
 1
 0
 1
 1
 1
 1
 1
 0
 0

In [8]:
bern = Bernoulli(0.9)

x = rand(bern, 10)

10-element Vector{Bool}:
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1

In [9]:
bag(x::Bool) = x == 1 ? "A" : "B"
ball(y::Bool) = y == 1 ? "赤" : "白"

x = bag.(rand(bern, 10))

10-element Vector{String}:
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "B"

In [10]:
function sample()
    x = bag(rand(Bernoulli(1//2))) # 1//2 -> Rational
    
    μ = x=="A" ? 1//5 : 3//5
    
    y = ball(rand(Bernoulli(μ)))
    
    x, μ, y
end

sample (generic function with 1 method)

In [12]:
for _ in 1:10
    x, μ, y = sample()
    println("袋: $(x), 玉: $(y)")
end

袋: A, 玉: 白
袋: A, 玉: 白
袋: B, 玉: 白
袋: B, 玉: 白
袋: B, 玉: 赤
袋: B, 玉: 赤
袋: A, 玉: 白
袋: B, 玉: 白
袋: B, 玉: 白
袋: B, 玉: 赤


### 周辺確率の計算

In [16]:
maxiter = 100
result = []

for _ in 1:maxiter
    x, μ, y = sample()
    push!(result, y)
end

mean(result .== "赤")

0.38

In [17]:
maxiter = 1_000_000
result = []

for _ in 1:maxiter
    x, μ, y = sample()
    push!(result, y)
end

mean(result .== "赤")

0.399876

### 条件付き確率の計算

In [24]:
y_obs = "赤"

maxiter = 1_000_000
x_results = []

for _ in 1:maxiter
    x, μ, y = sample()
    y == y_obs && push!(x_results, x) # y==y_obsの時のみx_resultsにpush
end

println("p(x=A | y=赤) : approx = $(mean(x_results .== "A")) (exact=$(1/4))")
println("p(x=B | y=赤) : approx = $(mean(x_results .== "B")) (exact=$(3/4))")

p(x=A | y=赤) : approx = 0.24973332465282125 (exact=0.25)
p(x=B | y=赤) : approx = 0.7502666753471787 (exact=0.75)


### 複数のデータがある場合

In [39]:
function sample(N)
    x = bag(rand(Bernoulli(1//2)))
    μ = x=="A" ? 1//5 : 3//5
    
    y = ball.(rand(Bernoulli(μ), N))
    
    x, μ, y
end

sample (generic function with 2 methods)

In [60]:
maxiter = 10_000
y_obs = ["赤", "赤", "白"]
x_results = []

for _ in 1:maxiter
    x, μ, y = sample(3)
    y == y_obs && push!(x_results, x)
end

println("acceptance rate = $(length(x_results)/maxiter)")
println("p(x=A | y_1=赤, y_2=赤, y_3=白) : approx = $(mean(x_results .== "A")) (exact=$(2/11))")
println("p(x=B | y_1=赤, y_2=赤, y_3=白) : approx = $(mean(x_results .== "B")) (exact=$(9/11))")

acceptance rate = 0.088
p(x=A | y_1=赤, y_2=赤, y_3=白) : approx = 0.18295454545454545 (exact=0.18181818181818182)
p(x=B | y_1=赤, y_2=赤, y_3=白) : approx = 0.8170454545454545 (exact=0.8181818181818182)


In [53]:
maxiter = 10_000
y_obs = ["赤", "赤", "白"]
x_results = []

for _ in 1:maxiter
    x, μ, y = sample(3)
    # 赤玉の個数が一致する場合に受容 -> 順番に関係なくなるので、受容率が上がる -> 計算効率up
    sum(y .== "赤") == sum(y_obs .== "赤") && push!(x_results, x)
end

println("acceptance rate = $(length(x_results)/maxiter)")
println("p(x=A | y_1=赤, y_2=赤, y_3=白) : approx = $(mean(x_results .== "A")) (exact=$(2/11))")
println("p(x=B | y_1=赤, y_2=赤, y_3=白) : approx = $(mean(x_results .== "B")) (exact=$(9/11))")

acceptance rate = 0.2684
p(x=A | y_1=赤, y_2=赤, y_3=白) : approx = 0.17883755588673622 (exact=0.18181818181818182)
p(x=B | y_1=赤, y_2=赤, y_3=白) : approx = 0.8211624441132638 (exact=0.8181818181818182)
